In [ ]:

# Telegram端步驟 https://pixnashpython.pixnet.net/blog/post/32391757-%E3%80%90telegram-api%E3%80%91python
'''
步驟一：在Telegram搜尋欄輸入BotFather並打開

步驟二：輸入 /newbot

步驟三：輸入機器人顯示名稱

步驟四：輸入機器人可搜尋名稱

步驟五：取得TOKEN複製起來(PYTHON用)
'''
#########
# https://www.freecodecamp.org/chinese/news/how-to-create-a-telegram-bot-using-python/
!pip install pyTelegramBotAPI


# 自動獲取工作資料
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:



# 62 版 $$$$$$$$$$$$$$$$$$$$$$$$$$
# https://chatgpt.com/share/ebfd449c-14c5-4dcc-a669-40d8f16f30c1

# 自動獲取工作資料
# chatgpt monica
# https://chat.openai.com/share/57c5b600-4c6a-40d2-b140-dd4761d6e5cd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time  # 新增此行，用於控制請求間的等待時間


# TG
import os
import re
import telebot
from google.colab import userdata


# 搵老闆用
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException

















class JobScraper:
    # selenium setting
    def __init__(self, keyword):
        self.keyword = keyword
        self.chrome_options = Options()
        self.chrome_options.add_argument('--headless')
        self.chrome_options.add_argument('--no-sandbox')
        self.chrome_options.add_argument('--disable-dev-shm-usage')
        self.chrome_options.add_argument('--disable-gpu')
        self.driver = webdriver.Chrome(options=self.chrome_options)


    ### 找工作 找工作 找工作 找工作 找工作 ###
    def _自動獲取香港勞工處工作資料(self):
        driver = webdriver.Chrome(options=self.chrome_options)
        driver.get("https://www1.jobs.gov.hk/0/tc/jobseeker/jobsearch/joblist/")

        # 填寫關鍵字並點擊搜尋按鈕
        search_box = driver.find_element(By.ID, "simp_searchKeyword")
        search_box.send_keys(self.keyword)
        search_button = driver.find_element(By.ID, "btnSearch")
        search_button.click()

        顯總料數 = 0
        _每頁量 = 20
        all工料 = []

        while True:
            # 等待搜尋結果加載完成
            WebDriverWait(driver, 10).until(EC.url_changes("https://www1.jobs.gov.hk/0/tc/jobseeker/jobsearch/joblist/"))

            # 獲取搜尋結果的HTML
            page_source = driver.page_source

            # 解析HTML並提取工作資料
            soup = BeautifulSoup(page_source, "html.parser")
            content_div = soup.find("div", id="content-innerdiv")
            job_listings = content_div.find_all("tr", class_="bg-white")

            # 取得資料總數
            if 顯總料數 == 0:
                total_jobs = soup.find("div", class_="py-2 d-lg-none").strong.text.strip()
                print("總共資料數:", total_jobs)
                顯總料數=1

                總共資料數 = f'總共資料數: {total_jobs}\n *** 資料來源:香港勞工處 ***\n\n'

            for job in job_listings:
                職位 = job.find("span").find("span").text.strip()
                刊日 = job.find_next("div", class_="col mb-2 mb-lg-0").span.text.strip()
                薪酬 = job.find_all("div", class_="col mb-2 mb-lg-0")[1].span.text.strip()
                地點 = job.find_all("div", class_="col mb-2 mb-lg-0")[2].span.text.strip()
                經驗 = job.find_all("div", class_="col mb-2 mb-lg-0")[3].span.text.strip()
                程度 = job.find_all("div", class_="col mb-2 mb-lg-0")[4].span.text.strip()
                網址 = "https://www1.jobs.gov.hk" + job.find_next("a")["href"]

                工料 = f'{職位}\n{薪酬}\n{地點}\n{經驗}\n{程度}\n{網址}\n$$$$ $$$$'
                all工料.append(工料)

            # 點擊下一頁按鈕
            if int(total_jobs) > _每頁量:
                # 點擊下一頁按鈕
                next_page_button = driver.find_element(By.XPATH, '//*[@id="content-innerdiv"]/div[4]/div/a[3]')
                next_page_button.click()
                _每頁量 += 20
            else:

              all工料.append(總共資料數)
              break

        # 關閉瀏覽器
        driver.quit()

        return all工料


    # 自動獲取香港勞工處工作資料 \
    def _爬取CareerJetHK內容(self):
        driver = webdriver.Chrome(options=self.chrome_options)
        all工料 = []
        page_number = 1

        while True:
            # 設置當前頁面的URL
            url = f"https://www.careerjet.com.hk/%E6%8B%9B%E8%81%98?s={self.keyword}&l=%E9%A6%99%E6%B8%AF&p={page_number}"
            driver.get(url)

            # 等待搜尋結果加載完成
            try:
                WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="search-content"]/ul'))
                )
            except Exception as e:
                print(f"搜尋結果頁面加載失敗或無資料: {e}")
                break

            # 取得搜尋結果的HTML
            soup = BeautifulSoup(driver.page_source, "html.parser")

            # 抓取指定區塊的內容
            job_listings = soup.select('#search-content > ul > li')

            # 如果當前頁沒有資料，則結束爬取
            if not job_listings:
                print("已無更多資料，結束爬取。")
                break

            for job in job_listings:
                try:
                    # 內文網址
                    content_url = job.find('h2').find('a')['href']
                    內文網址 = "https://www.careerjet.com.hk" + content_url
                    # 工作標題
                    工作標題 = job.find('h2').find('a').text.strip()
                    # 公司名稱
                    公司名稱 = job.find('p').text.strip()
                    # 公司地址
                    公司地址 = job.find_all('ul')[1].find('li').text.strip()
                    # 簡介
                    簡介 = job.find('div').text.strip()

                    # 調整排列順序
                    content_data = {
                        "工作標題": 工作標題,
                        "公司名稱": 公司名稱,
                        "公司地址": 公司地址,
                        "簡介": 簡介,
                        "內文網址": 內文網址
                    }
                    工料 = f'{公司名稱}\n{工作標題}\n{公司地址}\n{簡介}\n{內文網址}\n$$$$ $$$$'

                    all工料.append(工料)
                except Exception as e:
                    print("錯誤:", e)
                    continue
            #print(f"第{page_number}頁資料已成功抓取。")
            page_number += 1  # 跳到下一頁
        # 關閉瀏覽器
        driver.quit()

        all工料.append(f'總共資料數: {len(all工料)}\n *** 資料來源:careerjet.com.hk ***\n\n')

        #return all工料

        if len(all工料) > 1:
            # 如果找到資料，回傳搜尋結果
            return all工料
        else:
            # 如果沒有找到資料，回傳 "沒有資料"
            return all工料[0]


    # 自動獲取careerjet.com.hk工作資料 \


    def 獲取所有工作資料(self):
        回1 = self._自動獲取香港勞工處工作資料()
        回2 = self._爬取CareerJetHK內容()
        return 回1 + 回2
    ### 找工作 找工作 找工作 找工作 找工作 ###



    ### 找老闆 找老闆 找老闆 找老闆 找老闆 ###
    # https://chatgpt.com/share/dd6041da-f9d8-464b-a44e-6d7ccb3adee4
    # https://chatgpt.com/share/0fcb7dec-9243-428c-8ae5-3b2531aef601
    def _搵要請人的老闆(self):
        self.driver.get("https://www1.jobs.gov.hk/0/tc/jobseeker/jobsearch/joblist/")

        # 填寫關鍵字並點擊搜尋按鈕
        search_box = self.driver.find_element(By.XPATH, '//*[@id="simp_searchKeyword"]')
        search_box.send_keys(self.keyword)
        search_button = self.driver.find_element(By.XPATH, '//*[@id="btnSearch"]')
        search_button.click()

        job_links = []
        while True:

            # 抓取位於 '//*[@id="content-innerdiv"]/div[3]/strong' 的數字
            element = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content-innerdiv"]/div[3]/strong'))
            )
            number_text = element.text
            number = int(number_text)  # 將抓取到的文字轉換為數字

            if number > 0:
                # 如果數字大於 0，繼續下一步
                WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="job_list_table"]'))
                )
            else: 
              break

            # 等待頁面加載完成 
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="job_list_table"]'))
            )

            # 獲取搜尋結果的HTML
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")

            # 提取每個職位的連結
            job_rows = soup.find('table', id="job_list_table").find_all('tr')
            for row in job_rows:
                link_tag = row.find('a', id=re.compile(r'\d+_orderNo_hyper'))
                if link_tag:
                    link = link_tag.get('href')
                    job_links.append("https://www1.jobs.gov.hk" + link)
                else:
                    print("未找到職位連結，略過該行。")

            # 檢查是否有下一頁按鈕，並嘗試點擊
            try:
                next_button_element = self.driver.find_element(By.XPATH, '//*[@id="content-innerdiv"]/div[4]/div/a[3]')
                if next_button_element.is_displayed() and next_button_element.is_enabled():
                    next_button_element.click()
                    time.sleep(2)  # 確保頁面完全加載
                else:
                    print("下一頁按鈕不可點擊，結束抓取。")
                    break
            except (ElementNotInteractableException, NoSuchElementException):
                print("未找到下一頁按鈕或按鈕無法互動，結束抓取。")
                break

        return job_links


    def _搵老闆的聯絡(self, job_url):
        self.driver.get(job_url)

        WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="jobOrderTable"]'))
        )

        page_source = self.driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        # 提取聯絡方式並檢查是否有聯絡方式
        contact_info = self._西損老闆的聯絡(soup.find('span', id="openupRemark").text.strip() if soup.find('span', id="openupRemark") else "無聯絡方式")
        if contact_info == "無聯絡方式":
            return None

        # 如果有聯絡方式，繼續提取其他資料
        job_details = {}
        job_details['公司名稱'] = soup.find('span', id="empName").text.strip() if soup.find('span', id="empName") else "無公司名稱"
        job_details['職位'] = soup.find('span', id="jobTitle").text.strip() if soup.find('span', id="jobTitle") else "無職位"
        job_details['聯絡方式'] = contact_info

        return _整字雜項(job_details['聯絡方式'],job_details['公司名稱'],job_details['職位'])














    def _西損老闆的聯絡(self, contact_text):
        # 提取電話號碼
        phone_numbers = re.findall(r'\b[456789]\d{7}\b', contact_text)
        if phone_numbers:
            #return f'https://wa.me/{phone_numbers[0]}?text='
            return phone_numbers[0]


        # 若無電話號碼則提取電子郵件
        email_match = re.search(r'[\w\.-]+@[\w\.-]+', contact_text)
        if email_match:
            return email_match.group(0)

        return "無聯絡方式"

    def _康力搵老闆(self):
        job_links = self._搵要請人的老闆()
        all老闆的聯絡 = []

        for link in job_links:
            job_details = self._搵老闆的聯絡(link)
            if job_details:  # 只有當有聯絡方式時才加入列表
                all老闆的聯絡.append(job_details)

        # 關閉瀏覽器
        self.driver.quit()

        all老闆的聯絡.append(f'🐳🐳🐳 總共資料數: {len(all老闆的聯絡)} ... 🐳🐳🐳')

        #return all老闆的聯絡

        if len(all老闆的聯絡) > 1:
            # 如果找到資料，回傳搜尋結果
            return all老闆的聯絡
        else:
            # 如果沒有找到資料，回傳 "沒有資料"
            return all老闆的聯絡[0]

    ### 找老闆 找老闆 找老闆 找老闆 找老闆 ###













def _整字雜項(聯絡方式,公司名稱,職位):

  # 分類聯絡並加字
  聯絡方式B = ''
  if '@' in 聯絡方式:
    聯絡方式B = f"mailto:{聯絡方式}?subject=尊敬的{公司名稱},康力人力資源顧問公司敬上&body="
  else:
    聯絡方式B = f'https://wa.me/{聯絡方式}?text='

  
  sell客文 = f"""{聯絡方式B}尊敬的{公司名稱}同事，您好，

    我是康力人力資源顧問公司的莫仔。

    注意到貴公司正在招聘{職位}，特此聯繫，希望為貴司提供專業的人力資源解決方案。

    煩請您將此信息轉達給貴公司的招聘負責人。若合作成功，我們將向您提供每月平均$500的介紹費，聊表謝意。

    康力人力資源顧問公司敬上

    ***

    尊敬的{公司名稱}，您好，我是康力人力資源顧問公司的莫仔。

    在當今競爭激烈的市場環境下，招聘合適的人才並非易事。

    招聘過程繁瑣，從刊登廣告、篩選簡歷、面試安排到最終錄用，每一步都需耗費大量時間和資源，這無疑分散了您和團隊對核心業務的精力。

    這些挑戰無需您獨自承擔。我們康力專注於為企業解決招聘難題，助您大幅降低行政成本。

    我們的服務包括：

    A.人才招聘廣告刊登：
      我們從刊登招聘廣告、篩選合適人才，到講解工作內容及資質要求，再到安排面試或試工，全程提供無憂服務。
    月費：$2,000。

    B.人力資源外包：
      我們可以以外判形式安排合適人才到貴公司上班，並可代辦勞工合約、勞工保險及強積金等所有手續。
    費用面議。

    C.人力資源定制解決方案：
      我們將根據貴公司的業務需求，量身打造最合適的招聘和人力資源管理策略，幫助您吸引並留住優秀人才。
    費用面議。

    我們的優勢：

    1.完善的人才數據：
      我們整合各企業反饋的員工能力，將所有員工資料保存到數據庫中，協助企業篩選出合適的員工。

    2.豐富的人才網絡：
      我們擁有涵蓋不同行業的招聘群組，求職會員超過2萬人，日活躍會員約3,000人，助您快速找到合適人才。

    3.豐富的配對經驗：
      我們曾與多家知名企業合作，如HKTV、惠康等，積累了豐富的經驗，深知如何滿足僱主和求職者的需求。

    選擇與我們合作，貴公司不僅能提高員工質素，提昇公司形象，還能顯著降低行政成本。我們期待與您攜手，共創成功的未來！

    如有招聘需求或想了解更多詳情，請隨時與我們聯絡：

    wa.me/85252718959/?text=莫仔我要查詢人力資源顧問

    期待您的來詢！

    康力人力資源顧問公司
    敬上
    
    https://98672794.github.io/康力人力資源顧問公司/************************"""

  sell客文 = sell客文.replace('\n', '%0A').replace(' ', '%20').replace('=莫', '%3D莫')
  return sell客文












# Telegram bot配置
def _TG多工機器人():
    bot = telebot.TeleBot(userdata.get('tg2'))

    ### 說明 /? ###
    @bot.message_handler(commands=['?'])
    def send_welcome(message):
        bot.reply_to(message, "\
        /$+工作名 例: /$在家工作 \n\
        [資料來源:香港勞工處、careerjet.com.hk]\n\
        ")
        print('🐳🐳🐳 正在說明 🐳🐳🐳')


    ### 搜尋工作 /$在家工作 ###
    @bot.message_handler(func=lambda message: re.match(r'^/\$', message.text))
    def handle_dollar_sign_commands(message):
        keyword = message.text.replace("/$", "")
        bot.reply_to(message, f"正在獲取 {keyword} 的工作資料...請稍候...")
        print(f'🐳🐳🐳 正在獲取 {keyword} 的工作資料.. 🐳🐳🐳')

        # 使用 JobScraper 類別來獲取工作資料
        scraper = JobScraper(keyword)
        all_data = scraper.獲取所有工作資料()
        _回覆對話(all_data,message,'chunk')


    ### 找老闆的聯絡 ###
    @bot.message_handler(func=lambda message: re.search(r'^\\%.+', message.text))
    def handle_dollar_sign_commands(message):
        keyword = message.text.replace("\\%", "")
        bot.reply_to(message, f"🐳🐳🐳 正在獲取要招聘 {keyword} 的老闆...請稍候(約15分鐘)... 🐳🐳🐳")
        print(f'🐳🐳🐳 正在獲取要招聘 {keyword} 的老闆請稍候(約15分鐘)... 🐳🐳🐳')

        # 來找老闆
        scraper = JobScraper(keyword)
        all_data = scraper._康力搵老闆()
        _回覆對話(all_data,message,'paragraph')


    ### all用回覆 ###
    def _回覆對話(內容, msg, mode):
        """
        mode: 
        - 'chunk'：按消息長度分割並發送。搜尋工作
        - 'paragraph'：按段落分開並發送。找老闆的聯絡
        """
        MAX_MESSAGE_LENGTH = 4096
        
        if mode == 'chunk':
            # 分割消息：將消息分割成更小的塊，每個塊的長度不超過Telegram API的限制
            chunks = []
            chunk = ''
            for data in 內容:
                if len(chunk) + len(data) + 1 < MAX_MESSAGE_LENGTH:
                    chunk += data + '\n'
                else:
                    chunks.append(chunk)
                    chunk = data + '\n'
            if chunk:
                chunks.append(chunk)
            
            for chunk in chunks:
                try:
                    print(chunk)
                    bot.reply_to(msg, chunk)
                    time.sleep(1)  # 每發送一個訊息後等待 1 秒，避免觸發 API 限制
                except telebot.apihelper.ApiTelegramException as e:
                    if e.result_json['error_code'] == 429:
                        retry_after = int(e.result_json['parameters']['retry_after'])
                        print(f"觸發 API 限制，需等待 {retry_after} 秒...")
                        time.sleep(retry_after)
                        bot.reply_to(msg, chunk)
                    else:
                        print(f"發送訊息時出現錯誤: {e}")
        
        elif mode == 'paragraph':

            # 確保內容是一個字串，如果是列表，則將其轉換為字串
            if isinstance(內容, list):
                內容 = "\n".join(內容)  # 將列表中的項目合併為一個字串，每個項目用換行符分隔

            # 將內容按段落拆分
            lines = 內容.split("************************")  # 根據兩個換行符分割段落

            for line in lines:
                if line.strip():  # 確保行不是空的
                    try:
                        print(line.strip())
                        bot.reply_to(msg, line.strip())
                        time.sleep(1)  # 每發送一段後等待 1 秒，避免觸發 API 限制
                    except telebot.apihelper.ApiTelegramException as e:
                        if e.result_json['error_code'] == 429:
                            retry_after = int(e.result_json['parameters']['retry_after'])
                            print(f"觸發 API 限制，需等待 {retry_after} 秒...")
                            time.sleep(retry_after)
                            print(line.strip())
                            bot.reply_to(msg, line.strip())
                        else:
                            print(f"發送訊息時出現錯誤: {e}")
    print('$$$ TG多工機器人已執行 $$$')
    bot.infinity_polling()

# Telegram bot配置 \




















if __name__ == "__main__":
    _TG多工機器人()

# 62 版 $$$$$$$$$$$$$$$$$$$$$$$$$$